<a href="https://colab.research.google.com/github/DanteTheCat26/ML/blob/main/task2_linreg_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ДЗ Линейная регрессия

В данном задании мы рассмотрим набор данных об учащихся, собранный в 2006 году в одной из школ Португалии. Данные представлены в неудобном для машинного обучения виде, и содержат мусор. Ваша задача &mdash; привести их к надлежащему виду и обучить на них простую модель.

Данные состоят из четырех файлов:
- data.csv &mdash; основная таблица с информацией о учащихся
- scores.csv &mdash; список финальных оценок по одному из предметов (20-балльная шкала переведенная в проценты)
- attendance.csv &mdash; таблица посещений занятий по этому предмету
- school_support.txt &mdash; список учащихся, которым оказывается финансовая поддержка

Ваша задача &mdash; построить модель для предсказания финальных оценок исходя из всех остальных данных и проверить качество ее работы с помощью кросс-валидации. В качестве алгоритма мы будем использовать линейную регрессию

Расшифровка столбцов в data.csv для справки:
- age &mdash; возраст
- Medu &mdash; уровень образования матери (по некоторой условной шкале)
- Fedu &mdash; уровень образования отца (по некоторой условной шкале)
- traveltime &mdash; время в пути до школы (1 – < 15 мин., 2 – от 15 до 30 мин., 3 – от 30 мин. to 1 ч.
или 4 – > 1 ч.)
- studytime &mdash; время, затрачиваемое на занятия вне школы (1 – < 2 ч., 2 – от 2 до 5 ч., 3 – от 5 до 10 ч. или 4 – > 10 ч.)
- famrel &mdash; насколько хорошие отношения в семье у учащегося (по некоторой условной шкале)
- freetime &mdash; количество свободного времени вне школы (по некоторой условной шкале)
- goout &mdash; время, затрачиваемое на общение с друзьями (по некоторой условной шкале)
- Dalc &mdash; количество употребления алкоголя в учебные дни (по некоторой условной шкале)
- Walc &mdash; количество употребления алкоголя в неучебные дни (по некоторой условной шкале)
- health &mdash; уровень здоровья (по некоторой условной шкале)
- sex_M &mdash; пол: мужской (1) или женский (0)
- address_U &mdash; живет ли учащийся в городе (1) или в пригороде (0)
- famsize_LE3 &mdash; размер семьи: не больше 3 человек (1) или больше (0)
- Pstatus_T &mdash; живут ли родители вместе (1) или отдельно (0)
- nursery &mdash; посещал ли учащийся детский сад
- plans_university &mdash; планирует ли учащийся поступать в университет (-1 или 1)
- past_failures &mdash; количество неудовлетворительных оценок по другим предметам ранее (от 0 до 4)

*Примечание. Несколько признаков в данных содержат ошибки/проблемы/некорректности. Эти проблемы нужно исправить. Для
проверки &mdash; всего в данных таких проблем четыре.*

### Задача 1: сломанный признак (а может и не один)
__(1 балл)__

Загрузите таблицу data.csv.

Найдите в данных сломанный признак (он не соответствует описанию) и исправьте его.

In [13]:
from google.colab import files
import pandas as pd
import numpy as np

# Загружаем файл
uploaded = files.upload()

# Получаем имя загруженного файла
file_name = list(uploaded.keys())[0]

# Загрузка данных
df = pd.read_csv(file_name)

# Разделяем слитный столбец на два отдельных
def split_plans_failures(value):
    if pd.isna(value) or value == '':
        return pd.Series([np.nan, np.nan])

    value_str = str(value)
    if value_str[0] == '-':
        plans = -1
        failures_str = value_str[1:]
    else:
        plans = 1
        failures_str = value_str

    try:
        failures = int(failures_str)
        return pd.Series([plans, failures])
    except:
        return pd.Series([np.nan, np.nan])

# Применяем разделение
df[['plans_university', 'past_failures']] = df['plans_universitypast_failures'].apply(split_plans_failures)

# Удаляем старый слитный столбец
df = df.drop('plans_universitypast_failures', axis=1)


# Исправление всех несоответствий на медиану корректных значений

# 1. Возраст (0-52)
valid_ages = df[df['age'].between(0, 52)]['age']
age_median = valid_ages.median()
df.loc[~df['age'].between(0, 52), 'age'] = age_median

# 2. Время в пути (1-4)
valid_traveltime = df[df['traveltime'].between(1, 4)]['traveltime']
traveltime_median = valid_traveltime.median()
df.loc[~df['traveltime'].between(1, 4), 'traveltime'] = traveltime_median

# 3. Время на учебу (1-4)
valid_studytime = df[df['studytime'].between(1, 4)]['studytime']
studytime_median = valid_studytime.median()
df.loc[~df['studytime'].between(1, 4), 'studytime'] = studytime_median

# 10. Пол (0 или 1)
valid_sex = df[df['sex_M'].isin([0, 1])]['sex_M']
sex_median = valid_sex.median()
df.loc[~df['sex_M'].isin([0, 1]), 'sex_M'] = sex_median

# 11. Адрес (0 или 1)
valid_address = df[df['address_U'].isin([0, 1])]['address_U']
address_median = valid_address.median()
df.loc[~df['address_U'].isin([0, 1]), 'address_U'] = address_median

# 12. Размер семьи (0 или 1)
valid_famsize = df[df['famsize_LE3'].isin([0, 1])]['famsize_LE3']
famsize_median = valid_famsize.median()
df.loc[~df['famsize_LE3'].isin([0, 1]), 'famsize_LE3'] = famsize_median

# 13. Статус родителей (0 или 1)
valid_pstatus = df[df['Pstatus_T'].isin([0, 1])]['Pstatus_T']
pstatus_median = valid_pstatus.median()
df.loc[~df['Pstatus_T'].isin([0, 1]), 'Pstatus_T'] = pstatus_median

# 14. Детский сад (0 или 1)
valid_nursery = df[df['nursery'].isin([0, 1])]['nursery']
nursery_median = valid_nursery.median()
df.loc[~df['nursery'].isin([0, 1]), 'nursery'] = nursery_median

# 15. Планы в университет (-1 или 1)
valid_plans = df[df['plans_university'].isin([-1, 1])]['plans_university']
plans_median = valid_plans.median()
df.loc[~df['plans_university'].isin([-1, 1]), 'plans_university'] = plans_median

# 16. Прошлые неудачи (0-4)
valid_failures = df[df['past_failures'].between(0, 4)]['past_failures']
failures_median = valid_failures.median()
df.loc[~df['past_failures'].between(0, 4), 'past_failures'] = failures_median

# Сохранение исправленных данных
df.to_csv('new_data.csv', index=False)

# Скачиваем результат
files.download('new_data.csv')

Saving data.csv to data.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Задача 2: пропуски в данных
__(1 балл)__

Проверьте, есть ли в данных пропуски (значения NaN). Замените все пропущенные значения на среднее значение этого признака по столбцу.

*Hint: изучите в pandas функции loc, isnull, а также передачу булевых массивов в качестве индексов.*

In [14]:
from google.colab import files
import pandas as pd
import numpy as np

# Загружаем файл
uploaded = files.upload()

# Получаем имя загруженного файла
file_name = list(uploaded.keys())[0]

# Загрузка данных
df = pd.read_csv(file_name)

# Заменяем все пропущенные значения на медиану по столбцу
for column in df.columns:
    if df[column].isnull().any():
        column_median = df[column].median()
        df.loc[df[column].isnull(), column] = column_median

# Сохранение исправленных данных
df.to_csv('new_data.csv', index=False)

# Скачиваем результат
files.download('new_data.csv')

Saving new_data(7).csv to new_data(7).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Задача 3: нормализация данных
__(1 балл)__

Нормализуйте данные любым способом

In [15]:
from google.colab import files
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Загружаем файл
uploaded = files.upload()

# Получаем имя загруженного файла
file_name = list(uploaded.keys())[0]

# Загрузка данных
df = pd.read_csv(file_name)

# Нормализация данных методом Min-Max
scaler = MinMaxScaler()
numeric_columns = df.select_dtypes(include=[np.number]).columns
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

# Сохранение нормализованных данных
df.to_csv('new_data.csv', index=False)

# Скачиваем результат
files.download('new_data.csv')

Saving new_data(8).csv to new_data(8).csv


/usr/local/lib/python3.12/dist-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/usr/local/lib/python3.12/dist-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Задача 4: кросс-валидация для исходных данных
__(1 балл)__

Загрузите файл scores.csv и протестируйте, как линейная регрессия предсказывает ответ сейчас (с помощью кросс-валидации).

Кроссвалидацию сделайте по 4 разбивкам. Выведите качество в каждом их разбиений.

*Hint: воспользуйтесь sklearn.linear_model и sklearn.model_selection.*

In [9]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, KFold

# Загрузка файла на Google Colab
uploaded = files.upload()

# Получаем имя загруженного файла
file_name = list(uploaded.keys())[0]
# Загружаем данные
data = pd.read_csv('scores.csv', header=None, names=['score'])

# Готовим данные для регрессии
X = np.arange(len(data)).reshape(-1, 1)  # индексы как фичи
y = data['score'].values  # оценки как таргет

# Создаем модель и кросс-валидацию
model = LinearRegression()
kf = KFold(n_splits=4, shuffle=True, random_state=42)

# Кросс-валидация с R² метрикой
cv_scores = cross_val_score(model, X, y, cv=kf, scoring='r2')

# Выводим качество для каждого разбиения
print("Качество модели по разбиениям (R² score):")
for i, score in enumerate(cv_scores):
    print(f"Разбиение {i+1}: {score:.4f}")

print(f"\nСреднее качество: {cv_scores.mean():.4f}")

IndexError: list index out of range

### Задача 5: полные данные
__(2 балла)__

Воспользуйтесь файлами attendance.csv и school_support.txt для того, чтобы добавить новые признаки в данные. Желательно по максимуму использовать возможности pandas для упрощения преобразований.

school_suport число в строке значит что i-ый школьник из исходной таблицы получал мат помощь (обратите внимание что строк в файле меньше, подумайте как правильно импортировать данные)

Добавьте данные таким образом, чтобы качество выросло

In [11]:
import pandas as pd
import numpy as np

# Загружаем файлы
scores = pd.read_csv('scores.csv', header=None, names=['score'])
attendance = pd.read_csv('attendance.csv', sep=';')

with open('school_support.txt', 'r') as f:
    support_indices = [int(line.strip()) for line in f]

# Создаем основной DataFrame с оценками
df = pd.DataFrame({'score': scores['score']})

# Обрабатываем посещаемость - преобразуем в числовой формат
attendance_numeric = attendance.replace({'+': 1, '': 0}).fillna(0)

# Создаем признаки из посещаемости
df['total_attendance'] = attendance_numeric.sum(axis=1)
df['attendance_rate'] = df['total_attendance'] / attendance_numeric.shape[1]
df['attendance_std'] = attendance_numeric.std(axis=1)
df['max_consecutive_attendance'] = attendance_numeric.apply(
    lambda x: max((x == 1).cumsum() - (x == 1).cumsum().where(x != 1).ffill().fillna(0)), axis=1
)

# Добавляем информацию о материальной помощи
df['received_support'] = 0
df.loc[support_indices, 'received_support'] = 1

# Создаем взаимодействующие признаки
df['attendance_with_support'] = df['total_attendance'] * df['received_support']
df['score_with_support_effect'] = df['score'] * (1 + 0.1 * df['received_support'])

# Аналитика по группам
analysis = pd.DataFrame({
    'statistic': [
        'total_students',
        'students_with_support',
        'avg_score_all',
        'avg_score_with_support',
        'avg_score_without_support',
        'avg_attendance_rate_all',
        'avg_attendance_rate_with_support',
        'avg_attendance_rate_without_support',
        'correlation_score_attendance',
        'correlation_score_support'
    ],
    'value': [
        len(df),
        df['received_support'].sum(),
        df['score'].mean(),
        df[df['received_support'] == 1]['score'].mean(),
        df[df['received_support'] == 0]['score'].mean(),
        df['attendance_rate'].mean(),
        df[df['received_support'] == 1]['attendance_rate'].mean(),
        df[df['received_support'] == 0]['attendance_rate'].mean(),
        df['score'].corr(df['attendance_rate']),
        df['score'].corr(df['received_support'])
    ]
})

# Сохраняем оба файла
df.to_csv('aboba52_features.csv', index=False)
analysis.to_csv('aboba52_analysis.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'scores.csv'

### Задача 6: борьба с выбросами
__(1.5 балла)__

Качество предсказания может ухудшаться, если в данных присутствуют корректные значения признаков (с точки зрения чтения данных и применения методов), но не соответствующие реальным объектам. Например, данные могли быть введены в неверном формате, а потом слишком грубо приведены к общему виду, из-за чего ошибка не была замечена.
Попробуем от такого избавиться &mdash; а для этого такие объекты нужно сначала найти. Конечно, нам еще недоступны многие продвинутые способы, но давайте попробуем обойтись простыми.

Первый способ это сделать &mdash; посмотреть для каждого признака на распределение его значений и проверить крайние значения на правдоподобность. (постройте гистограммы для признаков, как минимум для подозрительных)

*Hint 1: используйте функцию DataFrame.hist*

*Hint 2: в описании датасета выше есть информация, необходимая для восстановления правильных значений*

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from google.colab import files

# Загрузка файла на Google Colab
uploaded = files.upload()

# Получаем имя загруженного файла
file_name = list(uploaded.keys())[0]
print(f"Загружен файл: {file_name}")

# Загружаем данные
df = pd.read_csv(file_name)

# Настройка отображения графиков
plt.figure(figsize=(20, 15))

# Построение гистограмм для всех признаков
for i, column in enumerate(df.columns, 1):
    plt.subplot(4, 5, i)

    # Для числовых признаков строим гистограмму
    if df[column].dtype in [np.int64, np.float64]:
        # Заменяем пропущенные значения на -1 для визуализации
        data_to_plot = df[column].fillna(-1)
        plt.hist(data_to_plot, bins=20, alpha=0.7, edgecolor='black')
        plt.title(f'{column}\n(min:{df[column].min()}, max:{df[column].max()})')
        plt.xlabel(column)
        plt.ylabel('Frequency')

    # Для бинарных признаков строим bar plot
    elif df[column].nunique() <= 2:
        df[column].value_counts().plot(kind='bar', alpha=0.7)
        plt.title(f'{column}\n(values: {sorted(df[column].unique())})')
        plt.xlabel(column)
        plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Проверка уникальных значений для каждого признака
print("Уникальные значения для каждого признака:")
for column in df.columns:
    unique_vals = sorted(df[column].dropna().unique())
    print(f"{column}: {unique_vals}")

# Основная статистика
print("\nОсновная статистика:")
print(df.describe())

# Пропущенные значения
print("\nПропущенные значения:")
print(df.isnull().sum())

KeyboardInterrupt: Interrupted by user

__(1.5 балла)__

Другой простой способ найти выбросы &mdash; сделать предсказание и посчитать ошибку на каждом объекте по отдельности и посмотреть на объекты с наибольшей ошибкой. Обучите линейную регрессию (функция fit) и для каждого объекта посчитайте среднеквадратичное отклонение. Постройте гистограмму распределения ошибок. Посмотрите на гистограмму и удалите из выборки те объекты на которых ошибка слишком большая.

Обратите внимание, что просто удалять все объекты с высокой ошибкой нельзя &mdash; это, конечно, хороший способ добиться меньшей ошибки (на данной выборке), но одновременно вы ухудшите обобщающую способность алгоритма. Вместо этого вам нужно найти однозначно ошибочные записи и их исправить.

*Hint: возможно, все проблемы уже были найдены первым способом; для проверки &mdash; в сумме здесь нужно исправить 3 проблемы.*

Для поиска ошибки на одном отдельном обьекте придётся обучить линейную регрессию руками. Частичный пример, допишите код. Постройте гистограмму распределения ошибок

NameError: name 'file_content' is not defined

In [ ]:
# Your code here
# ...

### Финальное предсказание и отчёт (1 балл)

Проведите предсказание еще раз и сравните качество с исходным. Запишите свои наблюдения - как изменялось качество обучения модели при использовании разных модификаций данных.